# Chapter 22: Building a Production-Grade Application

This capstone chapter synthesizes every concept covered in this handbook into a cohesive, production-ready application. We will architect, implement, and deploy a **Task Management API**—a real-world service handling task creation, assignment, and tracking. This project demonstrates professional patterns including clean architecture, type-safe code, comprehensive testing, containerized deployment, and observability.

Rather than isolated code snippets, we will build a complete system where each component follows industry standards: domain-driven design for business logic, the repository pattern for data access, dependency injection for testability, and structured logging for observability. By the end, you will have a reference architecture applicable to microservices, web applications, and automation platforms.

## 22.1 Project Planning: Architecture and Design

Production applications begin not with code, but with clear requirements and architectural decisions that guide implementation. We adopt **Clean Architecture** (also called Hexagonal or Onion Architecture), which isolates business logic from frameworks and databases, making the system testable and adaptable.

### Requirements Analysis

**Functional Requirements:**
*   Users can create, read, update, and delete tasks
*   Tasks have priorities (low, medium, high), statuses (todo, in_progress, done), and due dates
*   Users can assign tasks to other users
*   System sends notifications for overdue tasks

**Non-Functional Requirements:**
*   API response time < 200ms (p95)
*   99.9% uptime
*   Horizontal scalability (stateless design)
*   Audit logging for compliance

### Architectural Decisions

**Monolith vs. Microservices:** We choose a modular monolith—single deployable unit with internal service boundaries. This avoids distributed system complexity while maintaining clear separation of concerns. If specific modules require scaling later, they can be extracted.

**Framework Selection:** FastAPI provides automatic OpenAPI documentation, type validation via Pydantic, and async support, making it ideal for modern Python APIs.

**Data Storage:** PostgreSQL for relational data (ACID guarantees for task consistency) with SQLAlchemy 2.0 (modern ORM with type hints).

**Project Structure (Src Layout):**
```
taskmanager/
├── src/
│   └── taskmanager/          # Main package
│       ├── domain/           # Business logic (framework-agnostic)
│       │   ├── models.py     # Domain entities
│       │   └── services.py   # Business rules
│       ├── application/      # Use cases (orchestration)
│       │   ├── dto.py        # Data Transfer Objects
│       │   └── task_service.py
│       ├── infrastructure/   # External concerns
│       │   ├── persistence/  # Database implementations
│       │   ├── web/          # FastAPI endpoints
│       │   └── config.py     # Settings management
│       └── main.py           # Application entry point
├── tests/
│   ├── unit/                 # Domain logic tests
│   ├── integration/          # Database/API tests
│   └── conftest.py           # Pytest fixtures
├── docker-compose.yml
├── Dockerfile
└── pyproject.toml
```

**The Dependency Rule:** Dependencies point inward. The domain layer knows nothing of FastAPI or PostgreSQL; the infrastructure layer depends on the domain. This allows swapping PostgreSQL for SQLite in tests without touching business logic.

## 22.2 Implementation: Clean, Typed, and Tested Code

We implement the application layer by layer, starting from the domain (innermost) and moving outward to infrastructure.

### Domain Layer: Business Logic

The domain layer contains enterprise business rules independent of any technical implementation. These are pure Python classes with type hints.

```python
# src/taskmanager/domain/models.py
from datetime import datetime
from enum import Enum, auto
from typing import Optional
from dataclasses import dataclass, field


class TaskStatus(Enum):
    """Enumeration of possible task states."""
    TODO = "todo"
    IN_PROGRESS = "in_progress"
    DONE = "done"


class Priority(Enum):
    """Task priority levels."""
    LOW = 1
    MEDIUM = 2
    HIGH = 3


@dataclass(frozen=True)  # Immutable value object
class TaskId:
    """
    Domain identifier using Value Object pattern.
    Prevents mixing Task IDs with User IDs or other integers.
    """
    value: int
    
    def __str__(self) -> str:
        return f"Task-{self.value}"


@dataclass
class Task:
    """
    Domain entity representing a task.
    
    Entities have identity (TaskId) that persists across state changes.
    Value objects (like Priority) are immutable and replaced, not modified.
    """
    id: Optional[TaskId]  # None for new tasks not yet persisted
    title: str
    description: str
    status: TaskStatus
    priority: Priority
    assignee_id: Optional[str]  # Reference to user in another service
    due_date: datetime
    created_at: datetime = field(default_factory=datetime.utcnow)
    updated_at: Optional[datetime] = None
    
    def __post_init__(self) -> None:
        """Domain validation enforced at construction."""
        if not self.title or len(self.title) < 3:
            raise ValueError("Title must be at least 3 characters")
        if self.due_date < self.created_at:
            raise ValueError("Due date cannot be in the past")
    
    def complete(self) -> None:
        """
        Domain method ensuring business rules.
        Encapsulates state transition logic.
        """
        if self.status == TaskStatus.DONE:
            raise ValueError("Task is already completed")
        self.status = TaskStatus.DONE
        self.updated_at = datetime.utcnow()
    
    def is_overdue(self) -> bool:
        """Business rule: overdue if past due_date and not done."""
        return datetime.utcnow() > self.due_date and self.status != TaskStatus.DONE
    
    def to_dict(self) -> dict:
        """Serialization for debugging (not for API response)."""
        return {
            "id": str(self.id) if self.id else None,
            "title": self.title,
            "status": self.status.value,
            "priority": self.priority.name,
            "overdue": self.is_overdue()
        }
```

**Explanation:** The `Task` class encapsulates business rules (validation in `__post_init__`, state machine in `complete()`). Using `frozen=True` for `TaskId` ensures value objects cannot be accidentally modified. Domain exceptions (`ValueError`) are part of the business logic—different from HTTP exceptions in the web layer.

### Application Layer: Use Cases

The application layer orchestrates domain objects to fulfill use cases (Create Task, Assign Task). It defines interfaces (protocols) for infrastructure dependencies, following the Dependency Inversion Principle.

```python
# src/taskmanager/application/dto.py
from datetime import datetime
from typing import Optional
from pydantic import BaseModel, Field, ConfigDict


class TaskCreateDTO(BaseModel):
    """
    Data Transfer Object for creating tasks.
    Separates API contract from Domain model.
    """
    model_config = ConfigDict(frozen=True)  # Immutable DTO
    
    title: str = Field(..., min_length=3, max_length=200)
    description: str = Field(default="", max_length=2000)
    priority: str = Field(default="MEDIUM", pattern="^(LOW|MEDIUM|HIGH)$")
    assignee_id: Optional[str] = None
    due_date: datetime
    
    def to_domain(self) -> "Task":
        """Factory method converting DTO to Domain entity."""
        from taskmanager.domain.models import Task, TaskStatus, Priority
        
        return Task(
            id=None,
            title=self.title,
            description=self.description,
            status=TaskStatus.TODO,
            priority=Priority[self.priority],
            assignee_id=self.assignee_id,
            due_date=self.due_date
        )


class TaskResponseDTO(BaseModel):
    """DTO for API responses."""
    id: int
    title: str
    status: str
    priority: str
    assignee_id: Optional[str]
    due_date: datetime
    is_overdue: bool
    
    model_config = ConfigDict(from_attributes=True)
```

```python
# src/taskmanager/application/interfaces.py
from typing import Protocol, Optional, List
from taskmanager.domain.models import Task, TaskId


class TaskRepository(Protocol):
    """
    Repository Protocol (interface) defining persistence contract.
    
    The domain depends on this abstraction, not concrete SQLAlchemy code.
    This enables testing with in-memory repositories.
    """
    
    async def save(self, task: Task) -> Task:
        """Persist new task or update existing."""
        ...
    
    async def get_by_id(self, task_id: TaskId) -> Optional[Task]:
        """Retrieve by ID or return None."""
        ...
    
    async def list_by_assignee(self, assignee_id: str) -> List[Task]:
        """Query by assignee."""
        ...
    
    async def delete(self, task_id: TaskId) -> bool:
        """Delete task. Returns True if existed."""
        ...
```

```python
# src/taskmanager/application/task_service.py
from typing import List, Optional
import logging
from taskmanager.domain.models import Task, TaskId, TaskStatus
from taskmanager.application.interfaces import TaskRepository
from taskmanager.application.dto import TaskCreateDTO, TaskResponseDTO

logger = logging.getLogger(__name__)


class TaskService:
    """
    Application Service coordinating use cases.
    Contains no business rules (those are in Domain), only workflow logic.
    """
    
    def __init__(self, repository: TaskRepository) -> None:
        """
        Dependency Injection: receives implementation of repository protocol.
        """
        self._repo = repository
    
    async def create_task(self, dto: TaskCreateDTO) -> TaskResponseDTO:
        """
        Use case: Create new task.
        
        1. Convert DTO to Domain entity (validation happens in domain)
        2. Persist via repository
        3. Convert back to DTO for response
        """
        try:
            task = dto.to_domain()
            saved_task = await self._repo.save(task)
            
            logger.info(f"Task created: {saved_task.id}")
            return self._to_response(saved_task)
            
        except ValueError as e:
            # Domain validation error - convert to application exception
            logger.error(f"Invalid task data: {e}")
            raise TaskCreationError(str(e)) from e
    
    async def complete_task(self, task_id: int) -> Optional[TaskResponseDTO]:
        """
        Use case: Mark task as complete.
        
        Demonstrates loading domain object, calling domain method, saving.
        """
        task = await self._repo.get_by_id(TaskId(task_id))
        if not task:
            return None
        
        # Domain logic encapsulates state transition rules
        task.complete()
        
        updated = await self._repo.save(task)
        logger.info(f"Task completed: {updated.id}")
        return self._to_response(updated)
    
    async def get_overdue_tasks(self) -> List[TaskResponseDTO]:
        """
        Use case: Find all overdue tasks.
        
        Note: In production, this might use a database query rather than
        loading all tasks into memory. Here we demonstrate domain logic usage.
        """
        # In real implementation, repository would have efficient query
        all_tasks = await self._repo.list_all()  # Assuming this method exists
        overdue = [t for t in all_tasks if t.is_overdue()]
        
        return [self._to_response(t) for t in overdue]
    
    def _to_response(self, task: Task) -> TaskResponseDTO:
        """Map domain entity to response DTO."""
        return TaskResponseDTO(
            id=task.id.value if task.id else 0,
            title=task.title,
            status=task.status.value,
            priority=task.priority.name,
            assignee_id=task.assignee_id,
            due_date=task.due_date,
            is_overdue=task.is_overdue()
        )


class TaskCreationError(Exception):
    """Application-specific exception."""
    pass
```

**Explanation:** `TaskService` contains **workflow logic** (orchestration) but no **business logic** (rules). The `Protocol` type hint allows passing any object implementing the interface—SQLAlchemy repository in production, dictionary-based repository in tests. This is dependency injection without frameworks, using Python's type system.

### Infrastructure Layer: Database Implementation

The infrastructure layer provides concrete implementations of domain interfaces. Here we implement the `TaskRepository` using SQLAlchemy 2.0 with async support.

```python
# src/taskmanager/infrastructure/persistence/models.py
from datetime import datetime
from sqlalchemy import String, DateTime, Enum as SQLEnum, Integer
from sqlalchemy.orm import Mapped, mapped_column, DeclarativeBase


class Base(DeclarativeBase):
    """SQLAlchemy base with type annotation support."""


class TaskORM(Base):
    """
    SQLAlchemy ORM model (persistence details).
    
    Separated from Domain model to allow database optimizations
    without polluting business logic.
    """
    __tablename__ = "tasks"
    
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    title: Mapped[str] = mapped_column(String(200), nullable=False)
    description: Mapped[str] = mapped_column(String(2000), default="")
    status: Mapped[str] = mapped_column(String(20), default="todo")
    priority: Mapped[int] = mapped_column(Integer, default=2)  # 1=LOW, 2=MEDIUM, 3=HIGH
    assignee_id: Mapped[Optional[str]] = mapped_column(String(100), nullable=True)
    due_date: Mapped[datetime] = mapped_column(DateTime(timezone=True))
    created_at: Mapped[datetime] = mapped_column(DateTime, default=datetime.utcnow)
    updated_at: Mapped[Optional[datetime]] = mapped_column(DateTime, nullable=True)
    
    def to_domain(self) -> "Task":
        """Convert ORM model to Domain entity."""
        from taskmanager.domain.models import Task, TaskId, TaskStatus, Priority
        
        return Task(
            id=TaskId(self.id),
            title=self.title,
            description=self.description,
            status=TaskStatus(self.status),
            priority=Priority(self.priority),
            assignee_id=self.assignee_id,
            due_date=self.due_date,
            created_at=self.created_at,
            updated_at=self.updated_at
        )
    
    @classmethod
    def from_domain(cls, task: "Task") -> "TaskORM":
        """Convert Domain entity to ORM model."""
        return cls(
            id=task.id.value if task.id else None,
            title=task.title,
            description=task.description,
            status=task.status.value,
            priority=task.priority.value,
            assignee_id=task.assignee_id,
            due_date=task.due_date,
            created_at=task.created_at,
            updated_at=task.updated_at
        )
```

```python
# src/taskmanager/infrastructure/persistence/repository.py
from typing import Optional, List
from sqlalchemy.ext.asyncio import AsyncSession
from sqlalchemy import select, delete as sql_delete
from taskmanager.domain.models import Task, TaskId
from taskmanager.application.interfaces import TaskRepository
from taskmanager.infrastructure.persistence.models import TaskORM


class SQLAlchemyTaskRepository(TaskRepository):
    """
    Concrete repository implementation using SQLAlchemy.
    
    Implements the Protocol defined in the application layer.
    Translates between Domain objects and ORM models.
    """
    
    def __init__(self, session: AsyncSession) -> None:
        self._session = session
    
    async def save(self, task: Task) -> Task:
        """Insert or update task."""
        orm_task = TaskORM.from_domain(task)
        
        if task.id is None:
            # New task - insert
            self._session.add(orm_task)
            await self._session.flush()  # Generate ID
            await self._session.refresh(orm_task)
        else:
            # Existing task - merge
            await self._session.merge(orm_task)
        
        await self._session.commit()
        return orm_task.to_domain()
    
    async def get_by_id(self, task_id: TaskId) -> Optional[Task]:
        """Query by primary key."""
        result = await self._session.execute(
            select(TaskORM).where(TaskORM.id == task_id.value)
        )
        orm_task = result.scalar_one_or_none()
        return orm_task.to_domain() if orm_task else None
    
    async def list_by_assignee(self, assignee_id: str) -> List[Task]:
        """Query by assignee with filtering."""
        result = await self._session.execute(
            select(TaskORM)
            .where(TaskORM.assignee_id == assignee_id)
            .order_by(TaskORM.due_date.asc())  # Soonest first
        )
        return [row.to_domain() for row in result.scalars().all()]
    
    async def delete(self, task_id: TaskId) -> bool:
        """Delete by ID."""
        result = await self._session.execute(
            sql_delete(TaskORM).where(TaskORM.id == task_id.value)
        )
        await self._session.commit()
        return result.rowcount > 0
```

**Explanation:** The Repository pattern isolates database specifics. The domain knows nothing of SQLAlchemy; the infrastructure handles the translation. The Repository pattern isolates database specifics. The domain knows nothing of SQLAlchemy; the infrastructure handles the translation between `Task` (domain) and `TaskORM` (persistence). This mapping layer allows the domain to evolve independently—for example, if we refactor `Task` to use value objects, only the `to_domain()` and `from_domain()` methods change.

### Infrastructure Layer: Web Interface

The web layer adapts HTTP requests to application use cases. FastAPI's dependency injection system integrates cleanly with our architecture.

```python
# src/taskmanager/infrastructure/web/dependencies.py
from typing import AsyncGenerator
from fastapi import Depends, HTTPException, status
from sqlalchemy.ext.asyncio import AsyncSession

from taskmanager.infrastructure.config import get_settings, Settings
from taskmanager.infrastructure.persistence.database import get_session
from taskmanager.infrastructure.persistence.repository import SQLAlchemyTaskRepository
from taskmanager.application.task_service import TaskService


async def get_db_session() -> AsyncGenerator[AsyncSession, None]:
    """
    FastAPI dependency yielding database sessions.
    Ensures session cleanup after request.
    """
    async for session in get_session():
        yield session


async def get_task_service(
    session: AsyncSession = Depends(get_db_session)
) -> TaskService:
    """
    Factory injecting repository implementation into service.
    
    FastAPI resolves Depends(get_db_session) first, then passes it here.
    This wires the concrete SQLAlchemy repository to the application service.
    """
    repository = SQLAlchemyTaskRepository(session)
    return TaskService(repository)


def get_config() -> Settings:
    """Inject configuration settings."""
    return get_settings()
```

```python
# src/taskmanager/infrastructure/web/routes.py
from typing import List
from fastapi import APIRouter, Depends, HTTPException, status
import structlog

from taskmanager.application.dto import TaskCreateDTO, TaskResponseDTO
from taskmanager.application.task_service import TaskService, TaskCreationError
from taskmanager.infrastructure.web.dependencies import get_task_service

router = APIRouter(prefix="/tasks", tags=["tasks"])
logger = structlog.get_logger()


@router.post(
    "/", 
    response_model=TaskResponseDTO, 
    status_code=status.HTTP_201_CREATED,
    summary="Create a new task"
)
async def create_task(
    dto: TaskCreateDTO,
    service: TaskService = Depends(get_task_service)
) -> TaskResponseDTO:
    """
    Endpoint creating tasks.
    
    FastAPI automatically:
    - Validates request body against TaskCreateDTO schema
    - Injects TaskService via dependency
    - Serializes response to TaskResponseDTO JSON
    
    Args:
        dto: Validated request body
        service: Injected business logic service
        
    Returns:
        Created task with generated ID
    """
    try:
        return await service.create_task(dto)
    except TaskCreationError as e:
        # Convert domain/application exception to HTTP exception
        logger.error("Task creation failed", error=str(e))
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail=str(e)
        )


@router.get(
    "/{task_id}", 
    response_model=TaskResponseDTO,
    summary="Retrieve task by ID"
)
async def get_task(
    task_id: int,
    service: TaskService = Depends(get_task_service)
) -> TaskResponseDTO:
    """
    Retrieve specific task.
    
    Path parameter `task_id` is validated as integer automatically.
    """
    from taskmanager.domain.models import TaskId
    
    task = await service._repo.get_by_id(TaskId(task_id))
    if not task:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Task {task_id} not found"
        )
    
    # Note: In production, expose service methods instead of accessing repo directly
    return service._to_response(task)


@router.post(
    "/{task_id}/complete",
    response_model=TaskResponseDTO,
    summary="Mark task as complete"
)
async def complete_task(
    task_id: int,
    service: TaskService = Depends(get_task_service)
) -> TaskResponseDTO:
    """
    Complete a task endpoint.
    
    Demonstrates state transition through application service.
    """
    result = await service.complete_task(task_id)
    if not result:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Task not found"
        )
    return result
```

**Explanation:** The web layer is intentionally thin—it validates input (via Pydantic/DTOs), calls application services, and handles HTTP-specific concerns (status codes, headers). Business logic remains in the domain/application layers. The `Depends` system enables testability: tests can override `get_task_service` to inject mocked repositories without changing route code.

### Application Entry Point

The main module wires everything together, configuring logging, database connections, and the FastAPI application.

```python
# src/taskmanager/main.py
import logging
import sys
from contextlib import asynccontextmanager

import structlog
from fastapi import FastAPI
from sqlalchemy.ext.asyncio import create_async_engine

from taskmanager.infrastructure.config import get_settings
from taskmanager.infrastructure.web.routes import router as task_router
from taskmanager.infrastructure.persistence.database import init_db


def configure_logging() -> None:
    """
    Structured logging configuration for production.
    Uses structlog for JSON output in production, pretty console in dev.
    """
    settings = get_settings()
    
    # Configure standard library logging
    logging.basicConfig(
        format="%(message)s",
        stream=sys.stdout,
        level=logging.INFO if not settings.DEBUG else logging.DEBUG,
    )

    # Configure structlog processors
    structlog.configure(
        processors=[
            structlog.stdlib.filter_by_level,
            structlog.stdlib.add_logger_name,
            structlog.stdlib.add_log_level,
            structlog.stdlib.PositionalArgumentsFormatter(),
            structlog.processors.TimeStamper(fmt="iso"),
            structlog.processors.StackInfoRenderer(),
            structlog.processors.format_exc_info,
            structlog.processors.UnicodeDecoder(),
            structlog.processors.JSONRenderer() if not settings.DEBUG 
            else structlog.dev.ConsoleRenderer()
        ],
        context_class=dict,
        logger_factory=structlog.stdlib.LoggerFactory(),
        wrapper_class=structlog.stdlib.BoundLogger,
        cache_logger_on_first_use=True,
    )


@asynccontextmanager
async def lifespan(app: FastAPI):
    """
    Application lifespan events.
    
    Runs startup code before accepting requests, cleanup after.
    """
    # Startup
    settings = get_settings()
    engine = create_async_engine(settings.DATABASE_URL)
    await init_db(engine)
    
    yield  # Application runs here
    
    # Shutdown cleanup
    await engine.dispose()


def create_application() -> FastAPI:
    """
    Application factory pattern.
    
    Allows creating multiple app instances (useful for testing).
    """
    configure_logging()
    
    app = FastAPI(
        title="Task Manager API",
        description="Production-grade task management system",
        version="1.0.0",
        lifespan=lifespan,
        docs_url="/docs" if get_settings().DEBUG else None,  # Disable docs in prod
    )
    
    # Include routers
    app.include_router(task_router)
    
    # Health check endpoint
    @app.get("/health", tags=["health"])
    async def health_check():
        """Kubernetes/Load balancer health check endpoint."""
        return {"status": "healthy", "version": "1.0.0"}
    
    return app


# Global instance for uvicorn/gunicorn
app = create_application()


if __name__ == "__main__":
    import uvicorn
    uvicorn.run(
        "taskmanager.main:app",
        host="0.0.0.0",
        port=8000,
        reload=get_settings().DEBUG,
        log_config=None  # Use structlog
    )
```

**Explanation:** The factory pattern (`create_application`) enables creating differently configured apps for testing vs. production. The `lifespan` context manager handles async database initialization, which is cleaner than the older `on_event("startup")` approach. Structured logging (via `structlog`) ensures logs are parseable by log aggregation systems like ELK or Splunk in production.

## 22.3 Testing Strategy: Comprehensive Quality Assurance

Production code requires comprehensive testing: unit tests for domain logic, integration tests for database interactions, and end-to-end tests for API contracts.

### Unit Tests (Domain Logic)

```python
# tests/unit/test_domain.py
import pytest
from datetime import datetime, timedelta
from taskmanager.domain.models import Task, TaskId, TaskStatus, Priority


class TestTaskDomain:
    """
    Pure domain logic tests - no database, no HTTP, no mocking needed.
    Fast and deterministic.
    """
    
    def test_task_creation_valid(self):
        """Test valid task instantiation."""
        future = datetime.utcnow() + timedelta(days=1)
        task = Task(
            id=None,
            title="Test Task",
            description="Description",
            status=TaskStatus.TODO,
            priority=Priority.HIGH,
            assignee_id="user-123",
            due_date=future
        )
        
        assert task.title == "Test Task"
        assert task.status == TaskStatus.TODO
        assert task.is_overdue() is False
    
    def test_task_validation_short_title(self):
        """Test domain validation rejects invalid data."""
        with pytest.raises(ValueError, match="at least 3 characters"):
            Task(
                id=None,
                title="AB",  # Too short
                description="",
                status=TaskStatus.TODO,
                priority=Priority.MEDIUM,
                assignee_id=None,
                due_date=datetime.utcnow() + timedelta(days=1)
            )
    
    def test_task_complete_transition(self):
        """Test state machine logic."""
        task = self._create_task()
        assert task.status == TaskStatus.TODO
        
        task.complete()
        assert task.status == TaskStatus.DONE
        assert task.updated_at is not None
    
    def test_task_complete_already_done(self):
        """Test domain rule: cannot complete already completed task."""
        task = self._create_task()
        task.complete()
        
        with pytest.raises(ValueError, match="already completed"):
            task.complete()
    
    def test_overdue_detection(self):
        """Test business rule for overdue detection."""
        past = datetime.utcnow() - timedelta(days=1)
        task = Task(
            id=TaskId(1),
            title="Overdue",
            description="",
            status=TaskStatus.TODO,
            priority=Priority.LOW,
            assignee_id=None,
            due_date=past,
            created_at=past - timedelta(days=1)
        )
        
        assert task.is_overdue() is True
        
        # Completed tasks are not overdue even if past due date
        task.complete()
        assert task.is_overdue() is False
    
    def _create_task(self) -> Task:
        """Helper factory."""
        return Task(
            id=None,
            title="Valid Title",
            description="",
            status=TaskStatus.TODO,
            priority=Priority.MEDIUM,
            assignee_id=None,
            due_date=datetime.utcnow() + timedelta(days=1)
        )
```

**Explanation:** Domain tests use plain pytest with no fixtures or mocking. They verify business invariants: validation rules, state transitions, and calculations. These tests execute in milliseconds and provide the highest value—if domain logic is wrong, nothing else matters.

### Integration Tests (Database)

```python
# tests/integration/test_repository.py
import pytest
from datetime import datetime, timedelta
from sqlalchemy.ext.asyncio import AsyncSession, create_async_engine
from sqlalchemy.orm import sessionmaker

from taskmanager.infrastructure.persistence.database import Base
from taskmanager.infrastructure.persistence.repository import SQLAlchemyTaskRepository
from taskmanager.domain.models import Task, TaskId, TaskStatus, Priority


@pytest.fixture
async def db_session():
    """
    Fixture providing test database session.
    
    Uses in-memory SQLite for speed, though could use testcontainers 
    for PostgreSQL-specific features.
    """
    engine = create_async_engine("sqlite+aiosqlite:///:memory:")
    async with engine.begin() as conn:
        await conn.run_sync(Base.metadata.create_all)
    
    async_session = sessionmaker(
        engine, class_=AsyncSession, expire_on_commit=False
    )
    
    async with async_session() as session:
        yield session
    
    await engine.dispose()


class TestTaskRepository:
    """Integration tests verifying database interaction."""
    
    @pytest.mark.asyncio
    async def test_save_and_retrieve(self, db_session: AsyncSession):
        """Test round-trip persistence."""
        repo = SQLAlchemyTaskRepository(db_session)
        
        # Create domain object
        future = datetime.utcnow() + timedelta(days=1)
        task = Task(
            id=None,
            title="Integration Test",
            description="Testing SQLAlchemy repo",
            status=TaskStatus.TODO,
            priority=Priority.HIGH,
            assignee_id="user-456",
            due_date=future
        )
        
        # Save
        saved = await repo.save(task)
        assert saved.id is not None
        assert saved.id.value > 0
        
        # Retrieve
        retrieved = await repo.get_by_id(saved.id)
        assert retrieved is not None
        assert retrieved.title == "Integration Test"
        assert retrieved.priority == Priority.HIGH
    
    @pytest.mark.asyncio
    async def test_list_by_assignee(self, db_session: AsyncSession):
        """Test query operations."""
        repo = SQLAlchemyTaskRepository(db_session)
        
        # Create multiple tasks for different users
        for i in range(3):
            await repo.save(self._create_task(f"Task {i}", f"user-a"))
        
        await repo.save(self._create_task("Other user", "user-b"))
        
        # Query
        results = await repo.list_by_assignee("user-a")
        assert len(results) == 3
        assert all(t.assignee_id == "user-a" for t in results)
    
    def _create_task(self, title: str, assignee: str) -> Task:
        """Helper."""
        return Task(
            id=None,
            title=title,
            description="",
            status=TaskStatus.TODO,
            priority=Priority.MEDIUM,
            assignee_id=assignee,
            due_date=datetime.utcnow() + timedelta(days=1)
        )
```

**Explanation:** Integration tests verify that SQL queries work correctly, relationships load properly, and transactions commit. Using an in-memory database (`sqlite+aiosqlite`) keeps these fast, though for PostgreSQL-specific features (like JSONB or full-text search), use Testcontainers to spin up real database instances.

### API Tests (End-to-End)

```python
# tests/integration/test_api.py
import pytest
from httpx import AsyncClient
from fastapi import FastAPI

from taskmanager.main import create_application


@pytest.fixture
def app() -> FastAPI:
    """Create app with test configuration."""
    return create_application()


@pytest.mark.asyncio
async def test_create_task_endpoint(app: FastAPI):
    """Test full HTTP request/response cycle."""
    async with AsyncClient(app=app, base_url="http://test") as client:
        response = await client.post(
            "/tasks/",
            json={
                "title": "API Test Task",
                "description": "Testing the API",
                "priority": "HIGH",
                "due_date": "2025-12-31T23:59:59"
            }
        )
        
        assert response.status_code == 201
        data = response.json()
        assert data["title"] == "API Test Task"
        assert data["priority"] == "HIGH"
        assert "id" in data


@pytest.mark.asyncio
async def test_create_task_validation_error(app: FastAPI):
    """Test error handling."""
    async with AsyncClient(app=app, base_url="http://test") as client:
        response = await client.post(
            "/tasks/",
            json={
                "title": "AB",  # Too short
                "due_date": "2025-12-31T23:59:59"
            }
        )
        
        assert response.status_code == 422  # Validation error
```

## 22.4 Deployment: Production Configuration

### Docker Configuration

```dockerfile
# Dockerfile (Multi-stage for production)
FROM python:3.11-slim as builder

WORKDIR /app

# Install build dependencies
RUN apt-get update && apt-get install -y gcc libpq-dev && rm -rf /var/lib/apt/lists/*

# Install Python dependencies
COPY pyproject.toml .
RUN pip install --upgrade pip && \
    pip install poetry && \
    poetry config virtualenvs.create false && \
    poetry install --no-dev --no-interaction --no-ansi

FROM python:3.11-slim

WORKDIR /app

# Copy only necessary artifacts from builder
COPY --from=builder /usr/local/lib/python3.11/site-packages /usr/local/lib/python3.11/site-packages
COPY --from=builder /usr/local/bin /usr/local/bin

# Copy application code
COPY src/ ./src/

# Security: non-root user
RUN groupadd -r appuser && useradd -r -g appuser appuser
RUN chown -R appuser:appuser /app
USER appuser

# Health check
HEALTHCHECK --interval=30s --timeout=5s --start-period=5s --retries=3 \
    CMD python -c "import urllib.request; urllib.request.urlopen('http://localhost:8000/health')" || exit 1

EXPOSE 8000

CMD ["uvicorn", "taskmanager.main:app", "--host", "0.0.0.0", "--port", "8000", "--workers", "4"]
```

### Docker Compose (Production)

```yaml
# docker-compose.prod.yml
version: "3.8"

services:
  api:
    build:
      context: .
      dockerfile: Dockerfile
    environment:
      - DATABASE_URL=postgresql+asyncpg://postgres:secret@db:5432/taskmanager
      - DEBUG=0
      - SECRET_KEY=${SECRET_KEY}
    depends_on:
      db:
        condition: service_healthy
    restart: unless-stopped
    deploy:
      replicas: 2
      resources:
        limits:
          cpus: '0.5'
          memory: 512M

  db:
    image: postgres:15-alpine
    environment:
      POSTGRES_PASSWORD: secret
      POSTGRES_DB: taskmanager
    volumes:
      - postgres_data:/var/lib/postgresql/data
    healthcheck:
      test: ["CMD-SHELL", "pg_isready -U postgres"]
      interval: 5s
      timeout: 5s
      retries: 5
    restart: unless-stopped

  nginx:
    image: nginx:alpine
    ports:
      - "80:80"
      - "443:443"
    volumes:
      - ./nginx.conf:/etc/nginx/nginx.conf:ro
    depends_on:
      - api
    restart: unless-stopped

volumes:
  postgres_data:
```

## Summary

This capstone chapter demonstrated the construction of a production-grade Python application using Clean Architecture principles. You witnessed the separation of concerns between **Domain** (business rules), **Application** (use cases), and **Infrastructure** (frameworks and databases). Each layer depends only on abstractions defined in inner layers, creating a system that is testable, maintainable, and adaptable to changing requirements.

You implemented type-safe data transfer using **Pydantic DTOs**, leveraged **SQLAlchemy 2.0** for async database access, and exposed functionality via **FastAPI** with automatic dependency injection. The testing strategy covered unit tests for domain logic, integration tests for persistence, and end-to-end tests for API contracts. Finally, you containerized the application with multi-stage Docker builds for security and efficiency.

This architecture scales from small teams to large organizations. The modular structure allows extracting microservices when specific domains grow complex, while the dependency injection system enables testing without external services. By following these patterns—explicit over implicit, type safety over convenience, and separation of concerns over rapid hacking—you build Python applications that remain maintainable for years.

**End of Handbook**